In [1]:
from dataset import TranSiGenDataset
from model import TranSiGen
from utils import *
import pickle
import argparse
import warnings
warnings.filterwarnings('ignore')

SyntaxError: invalid syntax (model.py, line 231)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Carregamento dos dados

Os dados são carregados a partir de um arquivo `.h5`:

In [ ]:
data_path = '../data/LINCS2020/data_example/processed_data_id.h5'

In [ ]:
data = load_from_HDF(data_path)

O conteúdo do arquivo é um dicionário contendo as seguintes informações:

In [ ]:
print(", ".join(data.keys()))

- **LINCS_index**: 
- **canonical_smiles**: ids das moléculas dos compostos químicos. Esse id é convertido para o padrão Smiles usando o vetor `idx2smi` definido posteriormente.
- **cid**: cell line identifier
- **sig**: signature

Esse conjunto é, em resumo, um ponteiro para os dados. Os dados em si serão carregados mais à frente pela classe `TranSiGenDataset`.

## Definição das configurações gerais
Observações:
- `cell_count`: número de linhagens celulares. Usado apenas para identificar o modelo salvo.
- `feat_type`: tipo de representação das moléculas. Para o uso no modelo, o código Smiles é convertido para uma outra representação da molécula. Essa conversão pode ser feita tanto para um embedding pelo modelo pré-treinado KPGT, ou pela impressão digital molecular (*molecular fingerprint*) ECFP4.
- `split_type`: define como os dados serão dividos em treino, validação e teste. Os possíveis valores são
    - `random_split`: essa divisão é feita de forma aleatória.
    - `cell_split`: os conjuntos são dividos considerando uma mesma linhagem celular, para que não haja o risco de todos os dados de uma célula (ou boa parte deles) fiquem apenas em um dos conjuntos.
- `features_dim`: tamanho do vetor que representa a molécula. No caso do KPGT, esse valor é 2304. No caso do ECFP4, o valor é 2048.

In [ ]:
cell_count = len(set(data['cid']))
feat_type = 'KPGT'
batch_size = 64
learning_rate = 1e-3
beta = 0.1
dropout = 0.1
weight_decay = 1e-5
n_folds = 5
random_seed = 364039
split_type = 'smiles_split'
features_dim = 2304
features_embed_dim = [400]
n_latent = 100
init_mode = 'pretrain_shRNA'
n_epochs = 300
molecule_path = '../data/LINCS2020/idx2smi.pickle'

In [65]:
local_out = '../results/trained_models_{}_cell_{}/{}/feature_{}_init_{}/'.format(cell_count, split_type, random_seed, feat_type, init_mode)

Abaixo é carregador o vetor `idx2smi`, responsável por converter os índices das moléculas carregadas acima nos respectivos códigos Smiles.

In [66]:
with open(molecule_path, 'rb') as f:
    idx2smi = pickle.load(f)

Exemplo:

In [67]:
idx2smi[2]

'BrCC(=O)NCCc1ccc2ccccc2c1'

## Divisão de treino e teste

In [68]:
pair, pairv, pairt = split_data(data, n_folds=n_folds, split_type=split_type, rnds=random_seed)

In [69]:
train = TranSiGenDataset(
    LINCS_index=pair['LINCS_index'],
    mol_feature_type=feat_type,
    mol_id=pair['canonical_smiles'],
    cid=pair['cid']
)

valid = TranSiGenDataset(
    LINCS_index=pairv['LINCS_index'],
    mol_feature_type=feat_type,
    mol_id=pairv['canonical_smiles'],
    cid=pairv['cid']
)

test = TranSiGenDataset(
    LINCS_index=pairt['LINCS_index'],
    mol_feature_type=feat_type,
    mol_id=pairt['canonical_smiles'],
    cid=pairt['cid']
)

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4, worker_init_fn=seed_worker)
valid_loader = torch.utils.data.DataLoader(dataset=valid, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4, worker_init_fn=seed_worker)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4, worker_init_fn=seed_worker)

## Criação do Modelo

In [70]:
model = TranSiGen(
    n_genes=978,
    n_latent=n_latent,
    n_en_hidden=[1200],
    n_de_hidden=[800],
    features_dim=features_dim,
    features_embed_dim=features_embed_dim,
    init_w=True,
    beta=beta,
    device=dev,
    dropout=dropout,
    path_model=local_out,
    random_seed=random_seed
)
_ = model.to(dev)

### Arquitetura do Modelo
Arquitetura dos codificadores do $X_1$ e do $X_2$ (são iguais):

In [93]:
model.encoder_x1

Sequential(
  (0): Linear(in_features=978, out_features=1200, bias=True)
  (1): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
)

Arquitetura dos decodificadores do $X_1$ e do $X_2$ (também são iguais):

In [94]:
model.decoder_x2

Sequential(
  (0): Linear(in_features=100, out_features=800, bias=True)
  (1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
  (4): Linear(in_features=800, out_features=978, bias=True)
  (5): ReLU()
)

Arquitetura do embedder da molécula:

In [73]:
model.feat_embeddings

Sequential(
  (0): Linear(in_features=2304, out_features=400, bias=True)
  (1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.1, inplace=False)
)

## Treinamento/Carregamento do Modelo

In [97]:
import torch
import torch.nn.functional as F
from torch import nn, optim

In [268]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, x):
        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.input_dim ** 0.5)
        attention = self.softmax(scores)
        weighted = torch.bmm(attention, values)
        return weighted

In [370]:
class NoisePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.profile_encoder = nn.Sequential(
            nn.Linear(978, 100),
            nn.BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Tanh(),
            nn.Dropout(p=0.1, inplace=False),
        )
        self.molecule_encoder = nn.Sequential(
            nn.Linear(2304, 100),
            nn.BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Tanh(),
            nn.Dropout(p=0.1, inplace=False),
        )
        self.final_predictor = nn.Sequential(
            nn.Linear(200, 500),
            nn.BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.Tanh(),
            nn.Linear(500, 978),
            nn.ReLU(),
        )
        self.attention = SelfAttention(2)
    
    def forward(self, profile, molecule):
        prof_code = self.profile_encoder(profile)
        mol_code = self.molecule_encoder(molecule)
        code = torch.stack((prof_code, mol_code), dim=2)
        code = self.attention(code)
        code = code.view(-1, 200)
        prediction = self.final_predictor(code)
        return prediction

In [375]:
predictor = NoisePredictor().to(dev)
optimizer = optim.Adam(predictor.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [376]:
item = next(iter(train_loader))

In [377]:
for i in range(300):
    predictor.train()

    loss_value = 0
    for x1_train, x2_train, features, mol_id, cid, sig in train_loader:
        x1_train = x1_train.to(dev)
        x2_train = x2_train.to(dev)
        features = features.to(dev)
        
        if x1_train.shape[0] == 1:
            continue
        
        optimizer.zero_grad()
        
        x2_pred = predictor(x1_train, features)
        loss = F.mse_loss(x2_train, x2_pred, reduction="sum")

        loss_value += loss.item()
        loss.backward()
        optimizer.step()
    
    predictor.eval()
    with torch.no_grad():
        valid_loss_value = 0
        for x1_data, x2_data, mol_features, mol_id, cid, sig in valid_loader:
            data_x2_pred = predictor(x1_data.to(dev), mol_features.to(dev))
            valid_loss_value += F.mse_loss(x2_data.to(dev), data_x2_pred, reduction="sum").item()
        print(f"Epoch {i+1}. Train Loss {loss_value / len(train_loader)}. Valid Loss {valid_loss_value / len(valid_loader)}")

Epoch 1. Train Loss 2307784.25. Valid Loss 1343727.0
Epoch 2. Train Loss 2234564.75. Valid Loss 1311396.25
Epoch 3. Train Loss 2158085.5. Valid Loss 1272578.0
Epoch 4. Train Loss 2074556.875. Valid Loss 1231739.0
Epoch 5. Train Loss 1997476.0. Valid Loss 1191173.25
Epoch 6. Train Loss 1928593.5. Valid Loss 1151740.25
Epoch 7. Train Loss 1853993.5. Valid Loss 1113665.75
Epoch 8. Train Loss 1783243.625. Valid Loss 1076097.5
Epoch 9. Train Loss 1719605.625. Valid Loss 1039460.5
Epoch 10. Train Loss 1659318.25. Valid Loss 999664.875
Epoch 11. Train Loss 1610707.625. Valid Loss 957700.125
Epoch 12. Train Loss 1559925.875. Valid Loss 919547.0
Epoch 13. Train Loss 1507543.0. Valid Loss 884051.1875
Epoch 14. Train Loss 1466795.25. Valid Loss 856646.25
Epoch 15. Train Loss 1428699.5. Valid Loss 834972.625
Epoch 16. Train Loss 1397911.75. Valid Loss 821297.75
Epoch 17. Train Loss 1371658.75. Valid Loss 813588.625
Epoch 18. Train Loss 1347588.75. Valid Loss 811043.5
Epoch 19. Train Loss 1319266.5

In [384]:
results = {}
predictor.eval()
with torch.no_grad():
    for x1_data, x2_data, mol_features, mol_id, cid, sig in test_loader:
        x1_data = x1_data.to(dev)
        x2_data = x2_data.to(dev)
        delta_x = (x2_data - x1_data).data.cpu().numpy().astype(float)
        x2_pred = predictor(x1_data, mol_features.to(dev))
        delta_x_pred = (x2_pred - x1_data).data.cpu().numpy().astype(float)
        result = 0
        for i in range(delta_x.shape[0]):
            results[sig[i]] = get_metric_func('pearson')(delta_x[i, :], delta_x_pred[i, :])

In [385]:
results

{'A549_13': 0.1103346867166929,
 'PC3_6': 0.08399428564030627,
 'A549_6': -0.028775688029225685,
 'VCAP_9': 0.10153353105803228,
 'A375_6': 0.024216956296907496,
 'HA1E_6': 0.03853736720090515,
 'HA1E_9': 0.01084647689216178,
 'HT29_6': 0.028279612293074942,
 'VCAP_6': 0.08407544691831219,
 'HT29_9': 0.032128426369930976,
 'MCF7_9': 0.06877149542887703,
 'A549_9': 0.10518967302194415,
 'ASC_13': 0.12036072939785611,
 'A375_9': 0.05682655514783971,
 'MCF7_6': 0.04910575153115867,
 'HA1E_13': 0.028087619225809563,
 'PC3_9': 0.09485917464881406}

In [133]:
if init_mode == 'pretrain_shRNA':
    # Carregando modelo pré-treinado
    print('=====load vae for x1 and x2=======')
    model_dict = model.state_dict()
    filename = '../results/trained_model_shRNA_vae_x1/best_model.pt'
    model_base_x1 = torch.load(filename, map_location='cpu')
    model_base_x1_dict = model_base_x1.state_dict()
    for k in model_dict.keys():
        if k in model_base_x1_dict.keys():
            model_dict[k] = model_base_x1_dict[k]
    filename = '../results/trained_model_shRNA_vae_x2/best_model.pt'
    model_base_x2 = torch.load(filename, map_location='cpu')
    model_base_x2_dict = model_base_x2.state_dict()
    for k in model_dict.keys():
        if k in model_base_x2_dict.keys():
            model_dict[k] = model_base_x2_dict[k]
    model.load_state_dict(model_dict)
    del model_base_x1, model_base_x2
else:
    epoch_hist, best_epoch = model.train_model(
        train_loader=train_loader,
        test_loader=valid_loader,
        n_epochs=n_epochs,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        save_model=False
    )

=====load vae for x1 and x2=======


## Avaliação do Modelo no Conjunto Teste

In [84]:
_, _, test_metrics_dict_ls = model.test_model(loader=test_loader, metrics_func=['pearson', 'rmse', 'precision100'])

for name, rec_dict_value in zip(['test'], [test_metrics_dict_ls]):
    df_rec = pd.DataFrame.from_dict(rec_dict_value)
    smi_ls = []
    for smi_id in df_rec['cp_id']:
        smi_ls.append(idx2smi[smi_id])
    df_rec['canonical_smiles'] = smi_ls

In [90]:
df_rec.columns

Index(['x1_rec_pearson', 'x1_rec_rmse', 'x1_rec_neg_precision100',
       'x1_rec_pos_precision100', 'x2_rec_pearson', 'x2_rec_rmse',
       'x2_rec_neg_precision100', 'x2_rec_pos_precision100', 'x2_pred_pearson',
       'x2_pred_rmse', 'x2_pred_neg_precision100', 'x2_pred_pos_precision100',
       'DEG_rec_pearson', 'DEG_rec_rmse', 'DEG_rec_neg_precision100',
       'DEG_rec_pos_precision100', 'DEG_pred_pearson', 'DEG_pred_rmse',
       'DEG_pred_neg_precision100', 'DEG_pred_pos_precision100', 'cp_id',
       'cid', 'sig', 'canonical_smiles'],
      dtype='object')